In [1]:
import random
import numpy as np
import pandas as pd
import time
from scipy.stats import linregress
%timeit
from citipy import citipy
import requests
# Import the API key.
from config import weather_api_key
from datetime import datetime #Import the datetime module from the datelime library
import gmaps

In [2]:
#Create a set of random latitutes and longitudes - 2000 points
lats = np.random.uniform(low=-90.000,high = 90.000, size=2000)
lngs = np.random.uniform(low=-180.00, high=180.000, size=2000)
lat_lngs = zip(lats,lngs)

# Create a list for cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in lat_lngs:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)
#cities

739

In [3]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
url = ""
test = 0
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
city_data = []
for i, city in enumerate(cities):
    
    #Group cities in sets of 50
    if (i%50==0 and i>=50):
        set_count += 1
        record_count = 1
    
    #Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        #print(city_url)   
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":city_weather_desc,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print(f"City not found. Skipping...Set:{set_count}")
        #print(city_url)
        pass
    
#Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
City not found. Skipping...Set:1
City not found. Skipping...Set:1
City not found. Skipping...Set:1
City not found. Skipping...Set:1
City not found. Skipping...Set:1
City not found. Skipping...Set:1
City not found. Skipping...Set:2
City not found. Skipping...Set:3
City not found. Skipping...Set:3
City not found. Skipping...Set:3
City not found. Skipping...Set:3
City not found. Skipping...Set:3
City not found. Skipping...Set:4
City not found. Skipping...Set:4
City not found. Skipping...Set:4
City not found. Skipping...Set:4
City not found. Skipping...Set:4
City not found. Skipping...Set:5
City not found. Skipping...Set:5
City not found. Skipping...Set:5
City not found. Skipping...Set:5
City not found. Skipping...Set:5
City not found. Skipping...Set:5
City not found. Skipping...Set:5
City not found. Skipping...Set:6
City not found. Skipping...Set:6
City not found. Skipping...Set:6
City not found. Skipping...Set:7
City not found. 

In [4]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,Mar Del Plata,-38.0023,-57.5575,54.09,66,90,4.00,overcast clouds,AR,2021-09-27 01:18:41
1,Kapaa,22.0752,-159.3190,84.18,73,75,5.01,light rain,US,2021-09-27 01:19:10
2,Mataura,-46.1927,168.8643,51.48,35,6,2.44,clear sky,NZ,2021-09-27 01:18:39
3,Sao Filipe,14.8961,-24.4956,78.58,80,87,5.23,overcast clouds,CV,2021-09-27 01:19:10
4,Egvekinot,66.3167,-179.1667,26.69,71,9,2.80,clear sky,RU,2021-09-27 01:19:11
...,...,...,...,...,...,...,...,...,...,...
679,Zhigansk,66.7697,123.3711,36.27,74,87,22.84,overcast clouds,RU,2021-09-27 01:23:21
680,Shizuishan,39.2333,106.7694,65.41,46,100,10.25,overcast clouds,CN,2021-09-27 01:23:21
681,Vao,-22.6667,167.4833,68.36,63,19,4.74,few clouds,NC,2021-09-27 01:23:22
682,Temiscaming,46.7212,-79.0971,57.67,94,98,2.30,overcast clouds,CA,2021-09-27 01:23:22


In [5]:
new_column_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed","Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mar Del Plata,AR,2021-09-27 01:18:41,-38.0023,-57.5575,54.09,66,90,4.00,overcast clouds
1,Kapaa,US,2021-09-27 01:19:10,22.0752,-159.3190,84.18,73,75,5.01,light rain
2,Mataura,NZ,2021-09-27 01:18:39,-46.1927,168.8643,51.48,35,6,2.44,clear sky
3,Sao Filipe,CV,2021-09-27 01:19:10,14.8961,-24.4956,78.58,80,87,5.23,overcast clouds
4,Egvekinot,RU,2021-09-27 01:19:11,66.3167,-179.1667,26.69,71,9,2.80,clear sky
...,...,...,...,...,...,...,...,...,...,...
679,Zhigansk,RU,2021-09-27 01:23:21,66.7697,123.3711,36.27,74,87,22.84,overcast clouds
680,Shizuishan,CN,2021-09-27 01:23:21,39.2333,106.7694,65.41,46,100,10.25,overcast clouds
681,Vao,NC,2021-09-27 01:23:22,-22.6667,167.4833,68.36,63,19,4.74,few clouds
682,Temiscaming,CA,2021-09-27 01:23:22,46.7212,-79.0971,57.67,94,98,2.30,overcast clouds


In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")